In [1]:
import sys
import os

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [2]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [3]:
import malaya_speech
from malaya_speech.train.model import uis_rnn






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
import torch

In [5]:
torch.ones(10)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [6]:
tf.ones(10).shape

TensorShape([Dimension(10)])

In [7]:
tf.ones_initializer

tensorflow.python.ops.init_ops.Ones

In [8]:
tf.get_variable(
            name = 'sigma2',
            shape = [10],
            initializer = tf.ones_initializer(),
        )

<tf.Variable 'sigma2:0' shape=(10,) dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [9]:
import numpy as np

train_sequence = np.random.rand(1000, 100)
train_cluster_id = np.array(['A'] * 1000)

In [45]:
train_sequences = [
    np.random.rand(100, 100),
    np.random.rand(200, 100),
    np.random.rand(300, 100)]
train_cluster_ids = [
    np.array(['A'] * 100),
    np.array(['A'] * 200),
    np.array(['A'] * 300),]

In [46]:
uis_rnn.utils.estimate_transition_bias(train_cluster_ids)

(0.001669449081803005, 599)

In [47]:
concatenated_train_sequence, concatenated_train_cluster_id = uis_rnn.utils.concatenate_training_data(
         train_sequences,
         train_cluster_ids,
         True,
         True)

concatenated_train_cluster_id = np.array(concatenated_train_cluster_id)

In [48]:
concatenated_train_sequence.shape

(600, 100)

In [23]:
concatenated_train_sequence.shape

(1000, 100)

In [24]:
len(concatenated_train_cluster_id)

1000

In [25]:
sub_sequences, seq_lengths = uis_rnn.utils.resize_sequence(
        sequence=concatenated_train_sequence,
        cluster_id=concatenated_train_cluster_id,
        num_permutations=10)

In [28]:
len(sub_sequences)

10

In [33]:
num_clusters = len(seq_lengths)
sorted_seq_lengths = np.sort(seq_lengths)[::-1]
permute_index = np.argsort(seq_lengths)[::-1]

sorted_seq_lengths, permute_index

(array([1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001]),
 array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0]))

In [36]:
batch_size = 2
mini_batch = np.sort(np.random.choice(num_clusters, batch_size))
mini_batch

array([4, 4])

In [39]:
rnn_input = np.zeros(
    (sorted_seq_lengths[mini_batch[0]], batch_size, 100)
)
rnn_input.shape

(1001, 2, 100)

In [40]:
for i in range(batch_size):
    rnn_input[
        1 : sorted_seq_lengths[mini_batch[i]], i, :
    ] = sub_sequences[permute_index[mini_batch[i]]]

In [42]:
rnn_input.shape

(1001, 2, 100)

In [44]:
rnn_truth = rnn_input[1:, :, :]
rnn_truth.shape

(1000, 2, 100)